In [ ]:
import Data.String


In [ ]:
data Prop = Const Bool
    | Var   String
    | Not   Prop
    | And   Prop Prop
    | Imply Prop Prop
    deriving (Show,Read)
    
type Assoc k v  = [(k,v)]    
type Subst = Assoc Char Bool

In [ ]:
p1 :: Prop
p1 = Var "AA"

p2 :: Prop
p2  = And p1 p1 

In [ ]:
find :: Eq k=> k -> Assoc k v -> [v]
find k t = [ v |  (k',v)<-t , k==k'] 

In [ ]:
a = [('A',False),('B',True),('B',False)]
find 'B' a

---------

In [ ]:
1) get number of variables ! to know haw many processes we have 
2) (p1 `^>` p2) and (p1 `<>` p2) 
   (p1 startsAfter p2 ) and (p1 overlap p2)




In [ ]:
data TimeFrame  = TimeFrame Int Int  
                deriving (Show,Read)

getStart :: TimeFrame  -> Int
getStart   (TimeFrame start _) = start

getEnd :: TimeFrame  -> Int
getEnd    (TimeFrame _ end) = end


overlap ::  TimeFrame -> TimeFrame -> Bool
overlap x y | not ((getEnd x <  getStart y) || (getEnd y <  getStart x) ) = True
            | otherwise = False
           

In [2]:
mkTimeFrame :: Int -> Int -> TimeFrame  
mkTimeFrame start end
    | start <= end  = TimeFrame start end


safeMkTimeFrame :: Int -> Int -> Maybe TimeFrame  
safeMkTimeFrame start end
    | start <= end = Just (TimeFrame start end)
    | otherwise    = Nothing

In [3]:
startsAfter :: TimeFrame -> TimeFrame -> Bool
startsAfter x y | getStart x > getStart y = True
                | otherwise = False

startsBefore :: TimeFrame -> TimeFrame -> Bool
startsBefore x y = not (startsAfter x y)       

In [4]:
t1 = TimeFrame 1 4
t2 = TimeFrame 3 5
t3 = mkTimeFrame 1 3
t4 = mkTimeFrame 1 2

t1
t2
t3
t4
overlap t1 t2
overlap t3 t4
startsAfter  t1 t2
startsAfter  t3 t4
startsBefore t1 t2
startsBefore t3 t4

TimeFrame 1 4

TimeFrame 3 5

TimeFrame 1 3

TimeFrame 1 2

True

True

False

False

True

True

In [34]:
a = [(1,2),(3,4),(2,1)]

In [2]:
import Data.List (permutations)
--import Prettyprinter
import Data.Maybe 

a = [(1,2),(3,4),(2,1)]

split :: [a]->[(a,a)]
split x = zip [x!! y| y<- evens] [x!! y| y<-odds] 
     where 
       len   =   length x 
       odds  =  [1,3.. len]
       evens =  [0,2.. len]

lst2TimeFrames :: [(Int,Int)] -> [TimeFrame]
lst2TimeFrames  =  f3 . f2 . f1 
    where 
            f1 = map (uncurry safeMkTimeFrame) 
            f2 = filter isJust 
            f3 = map fromJust

lst2TimeFrames a

: 

In [6]:
import Data.List (permutations)
import Prettyprinter

myList = [1, 2, 3,4]
permutedLists = permutations myList

aa =  map lst2TimeFrames (map split permutedLists)
aa

Line 7: Use map once
Found:
map lst2TimeFrames (map split permutedLists)
Why not:
map (lst2TimeFrames . split) permutedLists

[[TimeFrame 1 2,TimeFrame 3 4],[TimeFrame 3 4],[TimeFrame 1 4],[TimeFrame 2 3,TimeFrame 1 4],[TimeFrame 2 4],[TimeFrame 1 3,TimeFrame 2 4],[],[TimeFrame 3 4],[],[],[TimeFrame 2 4],[TimeFrame 2 3],[TimeFrame 2 3],[TimeFrame 1 4,TimeFrame 2 3],[TimeFrame 1 2],[TimeFrame 1 3],[TimeFrame 2 4,TimeFrame 1 3],[],[],[TimeFrame 1 4],[TimeFrame 1 3],[TimeFrame 1 2],[TimeFrame 3 4,TimeFrame 1 2],[]]

In [8]:
import Data.String

type Assoc k v  = [(k,v)] 

find :: Eq k=> k -> Assoc k v -> v
find k t = head [ v |  (k',v)<-t , k==k'] 

type Subst = Assoc String TimeFrame    

-- Time Frame
data TimeFrame  = TimeFrame Int Int  deriving (Show,Read)
getStart :: TimeFrame  -> Int
getStart   (TimeFrame start _) = start

getEnd :: TimeFrame  -> Int
getEnd    (TimeFrame _ end) = end

overlap ::  TimeFrame -> TimeFrame -> Bool
overlap x y | not ((getEnd x <  getStart y) || (getEnd y <  getStart x) ) = True
            | otherwise = False
            
startsAfter :: TimeFrame -> TimeFrame -> Bool
startsAfter x y | getStart x > getStart y = True
                | otherwise = False

p1 :: ProcessModel 
p1 = And Overlap (Proc "procesA" Proc "procesB")  StartsBefore (Proc "procesA" Proc "procesB")

In [9]:
data ProcessOper  = Proc         String
                  | Overlap      ProcessOper ProcessOper
                  | StartsAfter  ProcessOper ProcessOper
                  | And          ProcessOper ProcessOper
                  | Or           ProcessOper ProcessOper
                  deriving (Show,Read)
                  
                  
eval :: Subst -> ProcessOper -> Bool

eval s (Overlap p q) = overlap (getTF p s) (getTF q s)
                     where 
                     subst (Proc x) = x 
                     getTF x = find (subst x) 

eval s (StartsAfter p q) = startsAfter (getTF p s) (getTF q s)
                     where 
                     subst (Proc x) = x 
                     getTF x = find (subst x) 

eval s (And p q) = eval s p && eval s q
eval s (Or p q)  = eval s p || eval s q


In [44]:
-- Testing 
                    


assocT = [("procesA" , TimeFrame 1 3) , ("procesB" , TimeFrame 4 5), ("procesC" , TimeFrame 3 5)]

p1 :: ProcessOper
p1 = And ( StartsAfter (Proc "procesB") (Proc "procesA")) 
         ( Overlap     (Proc "procesB") (Proc "procesC")) 


-- startsAfter (find "procesA" s) (find "procesB" s)
eval assocT p1

True

In [43]:
-- Getting Vars from ProcessOper
import Data.List (nub,sort) 

vars :: ProcessOper -> [String]
vars (Proc xs) = [xs]
vars (Overlap p q) = vars p ++ vars q
vars (StartsAfter p q) = vars p ++ vars q
vars (And p q) = vars p ++ vars q
vars (Or p q) = vars p ++ vars q
getvars :: ProcessOper -> [String]
getvars  = sort . nub . vars

getvars p1


["procesA","procesB","procesC"]

In [91]:
import Prettyprinter

multiplyString :: String -> Int -> String
multiplyString str n = concat (replicate n str)

tf2String :: TimeFrame -> String
tf2String (TimeFrame x y) =    multiplyString " " x  ++   multiplyString "*" (y-x)

assocTableCase = [("procesA" , TimeFrame 1 3) , ("procesB" , TimeFrame 4 5), ("procesC" , TimeFrame 4 7)]

p1 :: ProcessOper
p1 = And ( StartsAfter (Proc "procesB") (Proc "procesA")) 
         ( Overlap     (Proc "procesA") (Proc "procesC")) 

strCase :: [(String,TimeFrame)] -> String
strCase  = concatMap ((\x -> tf2String x ++"\n"). snd) 

showCase ::  [(String, TimeFrame)] -> Doc ann
showCase  = pretty . strCase

showCase assocTableCase

eval assocTableCase p1

 **
    *
    ***

False

In [84]:
:t showCase

showCase :: forall {ann}. [(String, TimeFrame)] -> Doc ann

---------

In [7]:
import IHaskell.Display
import Codec.Picture (generateImage, PixelRGBA8(..))

: 

In [11]:
{-# LANGUAGE OverloadedStrings #-}
-- cabal install http-conduit


import Network.HTTP.Simple


let plantUMLScript = "Bob -> Alice : Hello"
let remoteURL = "http://www.plantuml.com/plantuml/png/" ++ plantUMLScript
    
response <- httpLBS $ parseRequest_ remoteURL
let responseBody = getResponseBody response
writeFile "my_diagram.puml" responseBody


: 

In [33]:
{-# LANGUAGE OverloadedStrings #-}


plantUMLScript :: String
plantUMLScript = "@startuml\n Bob -> Alice : Helloo\n @enduml"

In [34]:
{-# LANGUAGE OverloadedStrings #-}

import Network.HTTP.Simple
import qualified Data.ByteString.Lazy as BL


let remoteURL = "http://www.plantuml.com/plantuml/png/~1" ++ plantUMLScriptx

response <- httpLBS $ parseRequest_ remoteURL
let responseBody = getResponseBody response

BL.writeFile "diagram.png" responseBody


: 

![title](./diagram.png?10 )